In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak
import time
from datetime import datetime
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
SEED = 10

In [3]:
print(f"Execution started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

Execution started at 2022-02-23 09:29:45.


In [4]:
filepath = "../Data Preprocessing/iot23_combined_2500k.csv"
df = pd.read_csv(filepath)

In [5]:
df

,Unnamed: 0,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,...,id.resp_p_23,id.resp_p_80,id.resp_p_123,id.resp_p_2323,id.resp_p_8080,id.resp_p_8081,id.resp_p_9527,id.resp_p_37215,id.resp_p_52869,id.resp_p_62336
0,70,2.999066,0,0,0,3,180,0,0,PartOfAHorizontalPortScan,...,1,0,0,0,0,0,0,0,0,0
1,79,0.000000,0,0,0,1,60,0,0,PartOfAHorizontalPortScan,...,1,0,0,0,0,0,0,0,0,0
2,83,2.998815,0,0,0,3,180,0,0,PartOfAHorizontalPortScan,...,1,0,0,0,0,0,0,0,0,0
3,97,0.000000,0,0,0,1,60,0,0,PartOfAHorizontalPortScan,...,1,0,0,0,0,0,0,0,0,0
4,101,0.000000,0,0,0,1,60,0,0,PartOfAHorizontalPortScan,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,2229370,0.000000,0,0,0,1,68,0,0,Benign,...,0,0,0,0,0,0,0,0,0,0
112646,2290883,0.000000,0,0,0,1,56,0,0,Benign,...,0,0,0,0,0,0,0,0,0,0
112647,2306017,0.000000,0,0,0,1,68,0,0,Benign,...,0,0,0,0,0,0,0,0,0,0
112648,2347856,0.000000,0,0,0,1,40,0,0,PartOfAHorizontalPortScan,...,1,0,0,0,0,0,0,0,0,0


In [6]:
del df['Unnamed: 0']

In [7]:
df['label'].value_counts()

DDoS                                             43892
PartOfAHorizontalPortScan                        33727
Benign                                           28219
Okiru                                             6788
C&C                                                  9
-   Malicious   C&C-HeartBeat-Attack                 9
-   Malicious   C&C-PartOfAHorizontalPortScan        3
C&C-FileDownload                                     1
C&C-HeartBeat-FileDownload                           1
Attack                                               1
Name: label, dtype: int64

In [8]:
good = 'Benign'
bad = 'PartOfAHorizontalPortScan'
filtered_labels = df['label'].value_counts().index.drop([good,bad])
for label in filtered_labels:
    df.drop(df[df.label == label].index, inplace=True)

In [9]:
df['label'].value_counts()

PartOfAHorizontalPortScan    33727
Benign                       28219
Name: label, dtype: int64

In [10]:
df.loc[(df.label == good), 'label'] = 0
df.loc[(df.label == bad), 'label'] = 1
df = df.astype({'label': int})

In [11]:
df['label'].value_counts()

1    33727
0    28219
Name: label, dtype: int64

In [12]:
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,id.resp_p_23,id.resp_p_80,id.resp_p_123,id.resp_p_2323,id.resp_p_8080,id.resp_p_8081,id.resp_p_9527,id.resp_p_37215,id.resp_p_52869,id.resp_p_62336
0,2.999066,0,0,0,3,180,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,0.000000,0,0,0,1,60,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,2.998815,0,0,0,3,180,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,0.000000,0,0,0,1,60,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,0.000000,0,0,0,1,60,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,0.000000,0,0,0,1,68,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
112646,0.000000,0,0,0,1,56,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
112647,0.000000,0,0,0,1,68,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
112648,0.000000,0,0,0,1,40,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [13]:
X = df.loc[:, df.columns != 'label']
print(X.shape)

(61946, 16031)


In [14]:
Y = df['label'].values
print(Y.shape)

(61946,)


In [15]:
from sklearn.feature_selection import SelectKBest, SelectPercentile, SelectFpr, SelectFdr, SelectFwe
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
X = MinMaxScaler(feature_range=(0,1)).fit_transform(X,Y)
X = SelectPercentile(percentile=1).fit_transform(X,Y)
print(X.shape)

C:\Users\marce\Anaconda2\envs\doutorado\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [    3   164   167   169   171   175   176   181   183   185   955  1634
  2327  2454  2747  2748  2750  2753  2755  2759  2766  2768  2771  2775
  2776  2779  2780  2783  2788  2790  2795  2796  2797  2799  2804  2805
  2806  2808  2811  2812  2813  2815  2816  2818  2820  2821  2826  2827
  2831  2833  2834  2836  2837  2839  2840  2841  2842  2843  2846  2847
  2849  2852  2853  2855  2858  2860  2862  2868  2870  2871  2874  2878
  2879  2884  2887  2889  2893  2895  2896  2899  2900  2901  2904  2908
  2913  2914  2915  2916  2922  2924  2926  2931  2932  2935  2939  2940
  2941  2942  2946  2948  2949  2954  2955  2956  2962  2963  2964  2966
  2967  2971  2972  2977  2979  2981  2982  2986  2988  2989  2991  2992
  2993  2994  2995  2996  2997  2998  2999  3004  3007  3017  3021  3023
  3026  3033  3037  3038  3039  3046  3049  3050  3056  3064 

(61946, 160)


In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=SEED, test_size=0.2)

In [17]:
model = ak.StructuredDataClassifier(overwrite=True, max_trials=5)

In [18]:
start = time.time()
print('program start...')
print()

history = model.fit(X_train, Y_train, validation_split=0.25)
print(history)

print()
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

Trial 5 Complete [00h 01m 44s]
val_accuracy: 0.9995150566101074

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 09m 32s
INFO:tensorflow:Oracle triggered exit
1549/1549 [==============================] - 4s 2ms/step - loss: 0.0657 - accuracy: 0.9825
INFO:tensorflow:Assets written to: .\structured_data_classifier\best_model\assets

program end...

time cost: 
713.64244556427 seconds


In [22]:
best_model = model.export_model()
print(best_model)

In [23]:
print("Evaluate on test data")
results = best_model.evaluate(X_test, Y_test)
print("test loss, test acc:", results)

Evaluate on test data
388/388 [==============================] - 2s 2ms/step - loss: 0.0031 - accuracy: 0.9993
test loss, test acc: [0.0031108807306736708, 0.9992735981941223]


In [24]:
print(f"Execution finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

Execution finished at 2022-02-23 09:50:49.
